#📊 Big Data na Gestão Pública:
#Investimentos, Dívidas e Renúncias Fiscais no Brasil

**Discplina: Tópicos de Big Data em Python - ARA0168**

**Professor: Vagner Cordeiro**

**Integrantes:**

**- Gabriel Passos**

**- Eduardo Stefanello**

**- Bernardo Moro**


**-------------------------------------------------------------------------------------------------------------------------------------------**

### **Partes Interessadas**
**🔹 Público-alvo: Sociedade em geral e órgãos públicos interessados em transparência dos investimentos.**

**🔹 Perfil: Cidadãos brasileiros, gestores públicos, pesquisadores (sem delimitação exata de idade ou escolaridade).**

**🔹 Parceiros: Dados públicos extraídos de portais oficiais (Tesouro Nacional, Receita Federal, etc).**

**-------------------------------------------------------------------------------------------------------------------------------------------**

### **Problemática Identificada**
**🔹 Falta de acessibilidade e interpretação dos dados públicos de investimentos, dívida e renúncia fiscal.**

**🔹 A população não tem clareza sobre como e onde o dinheiro público está sendo aplicado ao longo dos anos.**

**-------------------------------------------------------------------------------------------------------------------------------------------**

### **Justificativa**
**🔹 O projeto promove a transparência de dados públicos, usando tecnologias de Big Data e machine learning.**

**🔹 Relaciona-se com o curso por aplicar conhecimentos de dados, estatística em larga escala.**

**🔹 Transformar dados complexos em insights acessíveis com apoio de visualizações interativas.**

**-------------------------------------------------------------------------------------------------------------------------------------------**

### **Objetivos**
**🔹 Criar visualizações claras e interativas sobre investimentos, dívida e renúncia fiscal.**

**🔹 Aplicar modelos de previsão para auxiliar no planejamento e tomada de decisão.**

**🔹 Usar ciência de dados para fomentar a cidadania e controle social.**

**-------------------------------------------------------------------------------------------------------------------------------------------**

### **Plano de Trabalho**
**🔹 Dados: obtidos do portal `dados.gov`, com foco em dados abertos de investimentos, dívida pública e renúncia fiscal.**

**🔹 Limpeza de dados: realizada no Google Colab com Pandas. Após tratamento, os arquivos foram exportados em formato `.csv` e importados para o ambiente Databricks.**

**🔹 Integração: realizada com PySpark, unindo os dados anuais de investimento, dívida (dezembro) e renúncia, gerando uma base consolidada (`df_final_ano`).**

**🔹 Visualizações: desenvolvidas com `Plotly`, exibindo gráficos interativos para análise das tendências.**

**🔹 Modelos de Previsão (ML): uso do `Prophet` para prever séries temporais de investimento, dívida e renúncia até 2034.**

**🔹 Cronograma: planejamento e execução das tarefas registrado no `Trello`, com desenvolvimento entre abril e junho.**

**-------------------------------------------------------------------------------------------------------------------------------------------**

### **Grupo de Trabalho**

**🔹 Eduardo Stefanello e Gabriel Passos : Responsável por buscar os dados públicos e realizar a limpeza inicial no Google Colab. Criação do Roteiro Overleaf e do Repositório do GITHUB**

**🔹 Gabriel Passos: Responsável pela integração dos dados no Databricks, geração de visualizações interativas e desenvolvimento das previsões com o modelo Prophet.**

**🔹 Bernardo Moro: Responsável por compilar os dados tratados, organizar o material para apresentação e montar o cronograma no Trello.**


%md
**-------------------------------------------------------------------------------------------------------------------------------------------**

### **Metas e Indicadores**
**🔹 Dataset limpo e integrado.**

**🔹 Três previsões com `Prophet` funcionando (Investimentos, Dívida, Renúncia).**

**🔹 Dashboard interativo com gráficos.**

**🔹 Indicador: Clareza e compreensibilidade dos gráficos gerados (facilidade para o público interpretar os dados e projeções).**

**-------------------------------------------------------------------------------------------------------------------------------------------**

### **Recursos Previstos**
**🔹 Google Colab, Databricks Community, Plotly, Prophet, PySpark (sem custos).**

**🔹 Arquivos `.csv` públicos (sem custos).**

**-------------------------------------------------------------------------------------------------------------------------------------------**

### **Detalhamento Técnico**
**Etapas do Projeto:**

**🔹 Importação e limpeza dos dados: arquivos `.csv` tratados com Pandas no Colab e posteriormente processados com PySpark no Databricks.**

**🔹 Agregação anual: consolidação dos dados por ano com base nas colunas:**
**- `movimento_liquido_reais` (investimentos)**
**- `valor do estoque de dezembro` (dívida)**
**- `total anual` (renúncia)**

**🔹 Integração dos dados: junção dos três conjuntos em um único DataFrame (`df_final_ano`), usando `join` externo pela coluna `ano`.**

**🔹 Visualizações: gráficos interativos com `Plotly`, com destaque para a evolução temporal de cada indicador.**

**🔹 Modelos de Previsão com `Prophet`: modelos treinados separadamente para investimento, dívida e renúncia fiscal, projetando os valores de 2024 até 2034.**

**-------------------------------------------------------------------------------------------------------------------------------------------**

In [0]:
# Leitura dos dados (Spark + CSV)
df_invest = spark.read.option("header", True).option("sep", ",").csv("/FileStore/tables/investimentos_ok.csv")
df_divida = spark.read.option("header", True).option("sep", ",").csv("/FileStore/tables/divida_ok.csv")
df_renuncia = spark.read.option("header", True).option("sep", ",").csv("/FileStore/tables/renuncia_ok.csv")
df_cnae = spark.read.option("header", True).option("sep", ",").csv("/FileStore/tables/cnae_ok.csv")
df_baseagregada = spark.read.option("header", True).option("sep", ",").csv("/FileStore/tables/base_agregada_anual.csv")

In [0]:
from pyspark.sql.functions import year, month, col, to_date, sum as spark_sum

#coluna "mes do estoque" para o tipo data
df_divida = df_divida.withColumn("mes do estoque", to_date("mes do estoque"))

#valores da dívida para (double)
df_divida = df_divida.withColumn("valor do estoque", col("valor do estoque").cast("double"))

# extraindo o ano e o mês da data
df_divida = df_divida.withColumn("ano", year("mes do estoque"))
df_divida = df_divida.withColumn("mes", month("mes do estoque"))

# apenas os dados referentes a dezembro (mês 12), valor final do ano
df_divida_dez = df_divida.filter(col("mes") == 12)

# agrupando por ano e somando o valor do estoque da dívida
div_ano = df_divida_dez.groupBy("ano").agg(spark_sum("valor do estoque").alias("divida_total"))

df_invest = df_invest.withColumn("ano", year("data"))

#"f0000_ano" para "ano" renúncia fiscal
df_renuncia = df_renuncia.withColumnRenamed("f0000_ano", "ano")

# investimentos por ano e somando os valores
inv_ano = df_invest.groupBy("ano").agg(spark_sum("movimento_liquido_reais").alias("investimentos_total"))

# renúncia por ano e somando os valores
ren_ano = df_renuncia.groupBy("ano").agg(spark_sum("total").alias("renuncia_total"))

# join tabelas: investimentos, dívida e renúncia por ano
df_final_ano = inv_ano.join(div_ano, on="ano", how="outer") \
                      .join(ren_ano, on="ano", how="outer") \
                      .orderBy("ano")


**-------------------------------------------------------------------------------------------------------------------------------------------**

In [0]:
import plotly.graph_objects as go

#dataFrame spark para pandas e removendo linhas (NaN)
pdf = df_final_ano.dropna().toPandas()

#criando a figura do gráfico
fig = go.Figure()

#linha de Investimentos
fig.add_trace(go.Scatter(
    x=pdf["ano"], 
    y=pdf["investimentos_total"],
    mode='lines+markers',
    name='Investimentos',
    hovertemplate='Ano: %{x}<br>Investimentos: R$ %{y:,.2f}'
))

#linha da Dívida
fig.add_trace(go.Scatter(
    x=pdf["ano"], 
    y=pdf["divida_total"],
    mode='lines+markers',
    name='Dívida',
    hovertemplate='Ano: %{x}<br>Dívida: R$ %{y:,.2f}'
))

#linha da Renúncia Fiscal
fig.add_trace(go.Scatter(
    x=pdf["ano"], 
    y=pdf["renuncia_total"],
    mode='lines+markers',
    name='Renúncia',
    hovertemplate='Ano: %{x}<br>Renúncia: R$ %{y:,.2f}'
))

#layout do gráfico
fig.update_layout(
    title="📊 Evolução dos Valores Investimento / Dívida / Renúncia por Ano (2017–2023)",
    xaxis_title="Ano",
    yaxis_title="Valor (R$)",
    hovermode="x unified", #Tooltip unificada por eixo X
    template="plotly_white",
    width=950,
    height=500
)

fig.show()

### **📊 Evolução de Investimentos, Dívida e Renúncia Fiscal (2017–2023)**

**O gráfico mostra como o setor público brasileiro aplicou, acumulou e abriu mão de recursos nos últimos anos:**

**🔵 Investimentos: Em queda desde 2017, com forte retração em 2020 devido à pandemia. Recuperação tímida até 2023.**

**🔴 Dívida Pública: Crescimento contínuo, intensificado em 2020 com os gastos emergenciais da COVID-19. Em 2023, ultrapassa R$ 8 trilhões.**

**🟢 Renúncia Fiscal: Aumenta gradualmente, com saltos em 2020 e 2022, impulsionada por incentivos a setores estratégicos e medidas eleitorais.**

**👉 A comparação revela um Estado com alta dívida, baixo investimento e expansão de benefícios fiscais, refletindo desafios na gestão dos recursos públicos em momentos de crise e instabilidade econômica.**

**-------------------------------------------------------------------------------------------------------------------------------------------**

In [0]:
import plotly.express as px
from pyspark.sql.functions import sum

# agrupar os dados de investimento por estado (UF), excluindo os totais nacionais
pdf_uf = df_invest.filter(~df_invest["uf_desc"].isin(["BRASIL", "NACIONAL"])) \
    .groupBy("uf_desc") \
    .agg(sum("movimento_liquido_reais").alias("total_uf")) \
    .orderBy("total_uf", ascending=False) \
    .limit(10) \
    .toPandas()                                                            

# função para formatar os valores em Bilhões ou Trilhões
def format_valor(valor):
    if valor >= 1e12:
        return f"{valor / 1e12:.1f}T"
    else:
        return f"{valor / 1e9:.1f}B"

#aplicação da formatação nos valores totais
pdf_uf["texto_formatado"] = pdf_uf["total_uf"].apply(format_valor)

#gráfico de barras horizontais com Plotly
fig = px.bar(
    pdf_uf.sort_values("total_uf", ascending=True),
    x="total_uf",
    y="uf_desc",
    orientation="h",
    text="texto_formatado",
    title="💰 Top 10 Estados por Investimento Público (2014–2024)",
    labels={
        "total_uf": "Investimento Total (R$)",
        "uf_desc": "Estado (UF)"
    }
)

fig.update_layout(template="plotly_white")

fig.show()

### **💰 Top 10 Estados por Investimento Público (2014–2024)**

**O gráfico mostra os estados que mais receberam investimentos públicos no período de 2014 a 2024. Bahia, Rio Grande do Sul e Minas Gerais lideram, com valores que ultrapassam R$ 1 trilhão, indicando forte presença de projetos federais nessas regiões.**

**Estados como Santa Catarina, São Paulo e Paraná aparecem em seguida, com aportes acima de R$ 600 bilhões. A distribuição evidencia a concentração de recursos em determinadas regiões, o que pode refletir prioridades econômicas, estratégicas ou sociais ao longo da década.**

**-------------------------------------------------------------------------------------------------------------------------------------------**

In [0]:
import plotly.express as px
from pyspark.sql.functions import sum as spark_sum, col

df_mun = df_invest.filter(~df_invest["municipio"].isin(["BRASIL", "NACIONAL"]))

#agrupa por município, somando o valor de investimento total
pdf_mun = df_mun.groupBy("municipio") \
    .agg(spark_sum("movimento_liquido_reais").alias("total_municipio")) \
    .orderBy("total_municipio", ascending=False) \
    .limit(10).toPandas()

#converte o valor para bilhões para o gráfico
pdf_mun["total_municipio_bilhoes"] = pdf_mun["total_municipio"] / 1e9

fig = px.bar(
    pdf_mun,
    x="total_municipio_bilhoes",
    y="municipio",
    orientation="h",
    title="💰 Top 10 Municípios por Investimento Público (2014–2024)",
    labels={
        "total_municipio_bilhoes": "Investimento (Bilhões R$)",
        "municipio": "Município"
    }
)

# formata os rótulos para exibir os valores em bilhões com 1 casa decimal
fig.update_traces(texttemplate='%{x:.1f}B')

fig.update_layout(
    template="plotly_white",
    xaxis_title="Investimento Total (R$)",
    yaxis_title="Município",
    yaxis=dict(categoryorder='total ascending')
)

fig.show()

### **💰 Top 10 Municípios por Investimento Público (2014–2024)**

**Este gráfico apresenta os 10 municípios que mais receberam recursos públicos no período analisado. Brasília lidera com folga, refletindo seu papel como sede administrativa do país. Em seguida, aparecem grandes centros urbanos como Rio de Janeiro e Porto Alegre, evidenciando a concentração de investimentos em regiões metropolitanas com alta demanda por infraestrutura e serviços públicos.**

**A predominância de capitais e cidades estratégicas mostra a priorização de áreas com maior densidade populacional ou importância econômica para o país.**

**-------------------------------------------------------------------------------------------------------------------------------------------**

In [0]:
import plotly.express as px

df_regiao = df_invest.filter(~df_invest["regiao"].isin(["NACIONAL"]))

#agrupa os dados por região, somando o valor total de investimento
pdf_regiao = df_regiao.groupBy("regiao") \
    .agg(sum("movimento_liquido_reais").alias("total_regiao")) \
    .orderBy("total_regiao", ascending=False).toPandas()

fig = px.bar(
            pdf_regiao, x="regiao", y="total_regiao",
             title="💰 Top 5 Regiões por Investimento (2014-2024)",
             labels={"total_regiao": "Investimento Total (R$)", "regiao": "Região"},
             text_auto=True) # Exibe os valores automaticamente sobre as barras

fig.update_layout(template="plotly_white")

fig.show()

### **💰 Top 5 Regiões por Investimento (2014-2024)**

**O gráfico exibe os Top 5 investimentos públicos por região do Brasil entre 2014 e 2024. A Região Nordeste se destaca como líder em volume total de investimentos, ultrapassando 6 trilhões de reais, seguida pelas regiões Sudeste e Sul, com valores acima de 3 trilhões.**

**Apesar de concentrarem grandes centros econômicos, o Sudeste e o Sul receberam menos recursos que o Nordeste, o que pode estar relacionado a programas federais de incentivo ao desenvolvimento regional. O investimento no Exterior também aparece, embora em menor escala, refletindo ações pontuais de alcance internacional.**

**Esse recorte regional ajuda a compreender como os recursos públicos são distribuídos geograficamente, revelando prioridades territoriais nas políticas de investimento do governo federal.**

**-------------------------------------------------------------------------------------------------------------------------------------------**

In [0]:
import plotly.express as px

#agrupa os dados por tipo de função pública, somando o total de investimentos
pdf_funcao = df_invest.groupBy("funcao_desc") \
    .agg(sum("movimento_liquido_reais").alias("total_funcao")) \
    .orderBy("total_funcao", ascending=False) \
    .limit(10).toPandas()

fig = px.bar(pdf_funcao, x="total_funcao", y="funcao_desc", orientation="h",
             title="💰 Top 10 Funções Públicas por Investimento (2014-2024) ",
             labels={"total_funcao": "Investimento Total (R$)", "funcao_desc": "Função"},
             text_auto=True) # Mostra o valor em cima das barras
        
fig.update_layout(
            template="plotly_white",
            yaxis={'categoryorder': 'total ascending'})

fig.show()

### **💰 Top 10 Funções Públicas por Investimento (2014-2024)**

**O gráfico apresenta as 10 funções públicas que mais receberam investimentos entre 2014 e 2024. As áreas de Transporte e Defesa Nacional lideram com folga, somando juntas mais de 17 trilhões de reais. Essas prioridades podem refletir grandes obras de infraestrutura e gastos estratégicos com segurança e defesa.**

**Na sequência, aparecem Encargos Especiais (que incluem pagamento de juros da dívida), Educação, Urbanismo e Saúde, demonstrando a diversidade das frentes que absorvem recursos públicos.**

**A análise revela como o orçamento federal se distribui entre funções estruturais, sociais e operacionais, oferecendo uma visão macro das escolhas políticas ao longo da década.**

**-------------------------------------------------------------------------------------------------------------------------------------------**

In [0]:
import plotly.express as px
from pyspark.sql.functions import col, sum as spark_sum, row_number
from pyspark.sql.window import Window

#5 funções com maior investimento
top_funcoes = df_invest.groupBy("funcao_desc") \
    .agg(spark_sum("movimento_liquido_reais").alias("total_funcao")) \
    .orderBy(col("total_funcao").desc()) \
    .limit(5)

#filtra funções
top_funcoes_list = [row["funcao_desc"] for row in top_funcoes.collect()]
df_filtrado = df_invest.filter(col("funcao_desc").isin(top_funcoes_list))

#agrupa por programa
df_programas = df_filtrado.groupBy("funcao_desc", "programa_desc") \
    .agg(spark_sum("movimento_liquido_reais").alias("total_investido"))

#3 maiores programas por função
window = Window.partitionBy("funcao_desc").orderBy(col("total_investido").desc())
top_programas = df_programas.withColumn("rank", row_number().over(window)) \
                            .filter(col("rank") <= 3) \
                            .drop("rank")

pdf_programas = top_programas.toPandas()

#quebra de linha nos nomes longos
def quebra_linha(texto, max_chars=35):
    if len(texto) > max_chars:
        return texto[:max_chars] + "<br>" + texto[max_chars:]
    return texto

pdf_programas["programa_exibicao"] = pdf_programas["programa_desc"].apply(quebra_linha)

#formatar valores
def formatar_valor(v):
    if v >= 1e12:
        return f"{v / 1e12:.2f}T"
    else:
        return f"{v / 1e9:.2f}B"

pdf_programas["texto_formatado"] = pdf_programas["total_investido"].apply(formatar_valor)

fig = px.bar(pdf_programas,
             x="total_investido",
             y="programa_exibicao",
             color="funcao_desc",
             title="💰 Top 3 Programas das 5 Funções com Maior Investimento",
             labels={"total_investido": "Total Investido (R$)", "programa_exibicao": "Programa"},
             height=700,
             orientation="h")

fig.update_traces(text=pdf_programas["texto_formatado"])

fig.update_layout(template="plotly_white",
                  legend_title="Função",
                  yaxis=dict(categoryorder="total ascending"))
                  
fig.show()

### **💰 Top 3 Programas das 5 Funções com Maior Investimento**

**Este gráfico apresenta os principais programas públicos dentro das cinco funções que mais receberam investimentos no período analisado. Entre os destaques:**

**- Defesa Nacional lidera com um programa isolado que ultrapassa a marca de R$ 6 trilhões.**

**- A função Transporte aparece com diversos programas relevantes, como transporte rodoviário e mobilidade urbana.**

**- Educação mostra forte presença com iniciativas voltadas à qualidade do ensino básico e superior**.

**- As funções de Urbanismo e Encargos Especiais também possuem programas com valores expressivos, voltados à infraestrutura e obrigações constitucionais.**

**O gráfico permite observar quais áreas concentram os maiores recursos e quais programas específicos absorvem boa parte dos investimentos públicos.**

**-------------------------------------------------------------------------------------------------------------------------------------------**

In [0]:
from pyspark.sql.functions import col, sum
import plotly.express as px

#total para numérica
df_renuncia = df_renuncia.withColumn("total", col("total").cast("double"))

#agregar o total por empresa
top_empresas = df_renuncia.groupBy("razao_social") \
    .agg(sum("total").alias("renuncia_total")) \
    .orderBy(col("renuncia_total").desc()) \
    .limit(10)

pdf_empresas = top_empresas.toPandas()

fig = px.bar(pdf_empresas,
             x="renuncia_total",
             y="razao_social",
             orientation="h",
             title="🏢 Top 10 Empresas com Maior Renúncia Fiscal (2015-2023)",
             labels={"renuncia_total": "Total Renunciado (R$)", "razao_social": "Empresa"},
             text_auto=True)
             
fig.update_layout(template="plotly_white", yaxis={'categoryorder':'total ascending'})

fig.show()

### **🏢 Top 10 Empresas com Maior Renúncia Fiscal (2015–2023)**

**O gráfico apresenta as dez empresas que mais se beneficiaram de renúncias fiscais no período de 2015 a 2023. A VALE S/A lidera isoladamente, acumulando quase 30 bilhões de reais em incentivos. Em seguida, aparecem grandes corporações como PETROBRAS e GLOBO COMUNICAÇÃO.**

**Esses dados revelam a concentração de incentivos fiscais em grandes empresas, especialmente nos setores de mineração, energia e comunicação, levantando debates sobre a efetividade e o equilíbrio dessas políticas de renúncia.**

**-------------------------------------------------------------------------------------------------------------------------------------------**

In [0]:
from pyspark.sql.functions import regexp_replace, col, sum
import plotly.express as px

#agregação da renúncia fiscal total por CNAE
df_cnae_agg = df_renuncia.groupBy("cnae") \
    .agg(sum("total").alias("renuncia_total"))

#junção com descrição do CNAE
df_cnae_renuncia = df_cnae_agg.join(df_cnae, on="cnae", how="left")

#descrições nulas
df_cnae_renuncia = df_cnae_renuncia.fillna({'descricao': 'Descrição não disponível'})

top_cnaes = df_cnae_renuncia.orderBy(col("renuncia_total").desc()).limit(10).toPandas()

fig = px.bar(
    top_cnaes,
    x="renuncia_total",
    y="descricao",
    orientation="h",
    title="🏢 Top 10 Setores Econômicos com Maior Renúncia Fiscal",
    labels={"descricao": "Setor Econômico (CNAE)", "renuncia_total": "Renúncia Fiscal (R$)"},
    text_auto=True
)
fig.update_layout(
    template="plotly_white",
    yaxis=dict(categoryorder="total ascending"),
    height=600
)
fig.show()

### **🏢 Top 10 Setores Econômicos com Maior Renúncia Fiscal**

**O gráfico mostra os setores com maiores valores de renúncia fiscal entre 2015 e 2023. O setor de extração de minério de ferro lidera com folga, seguido por atividades de televisão aberta e geração de energia elétrica.**

**Esses dados revelam onde estão concentrados os incentivos tributários, com destaque para setores estratégicos da infraestrutura e da indústria extrativa. O investimento público indireto nesses setores pode estar relacionado à sua importância econômica ou à busca por estímulo em áreas essenciais ao desenvolvimento do país.**

**-------------------------------------------------------------------------------------------------------------------------------------------**

In [0]:
%pip install prophet

from prophet import Prophet
import pandas as pd
import plotly.graph_objects as go

df = df_baseagregada.toPandas().copy()

df["ano"] = df["ano"].astype(int)  #coluna "ano" inteiro
df["investimentos_total"] = df["investimentos_total"].astype(float)  #converte float
df["y"] = df["investimentos_total"] / 1e12  #converte para trilhões de reais

df["ds"] = pd.to_datetime(df["ano"].astype(str), format="%Y")  #cria coluna de data exigida Prophet

#treina o modelo Prophet com o dados
model = Prophet()
model.fit(df[["ds", "y"]])  #'ds' (data) e 'y' (valor a ser previsto)

#datas futuras até 2034
future = model.make_future_dataframe(periods=11, freq="Y")
forecast = model.predict(future)  #previsões

#separa dados reais (até 2024) previsões (a partir de 2025)
real = forecast[forecast["ds"] <= pd.to_datetime("2024-12-31")]
previsao = forecast[forecast["ds"] > pd.to_datetime("2024-12-31")]

fig = go.Figure()

#linha dos dados reais
fig.add_trace(go.Scatter(
    x=real["ds"], y=real["yhat"],
    mode="lines+markers",
    name="Investimento Real",
    line=dict(color="green")
))

#linha das previsões
fig.add_trace(go.Scatter(
    x=previsao["ds"], y=previsao["yhat"],
    mode="lines+markers",
    name="Previsão (2025–2034)",
    line=dict(color="blue", dash="dash")
))

fig.update_layout(
    title="📈 Previsão de Investimentos Públicos (2014–2034) com Prophet",
    xaxis_title="Ano",
    yaxis_title="Investimentos (Trilhões R$)",
    xaxis=dict(tickformat="%Y"),
    template="plotly_white",
    width=950,
    height=500
)

fig.show() 

### **📊 Previsão de Investimentos Públicos (2014–2034)**
### 
**Utilizando o modelo Prophet, foi realizada a projeção dos investimentos públicos anuais com base na série histórica de 2014 a 2024.**

**Os dados indicam uma recuperação nos últimos anos, e o modelo prevê uma tendência de crescimento moderado até 2034, com valores variando entre 5,1 e 5,8 trilhões de reais ao ano.**

**A análise foi feita sem variáveis externas, focando apenas na evolução histórica dos dados. O Prophet foi escolhido pela sua eficiência em séries temporais anuais e por capturar tendências mesmo em séries mais curtas.**

**-------------------------------------------------------------------------------------------------------------------------------------------**

In [0]:
from prophet import Prophet
import pandas as pd
import plotly.graph_objects as go

df = df_baseagregada.toPandas().copy()

df["ano"] = df["ano"].astype(int)  #coluna "ano" inteiro
df["divida_total"] = df["divida_total"].astype(float)   #converte float

#filtro dados a partir de 2017
df = df[df["ano"] >= 2017].copy()

df["y"] = df["divida_total"] / 1e12  #converte para trilhões
df["ds"] = pd.to_datetime(df["ano"], format="%Y") #cria coluna de data exigida Prophet

#cria e treina o modelo Prophet
model = Prophet()
model.fit(df[["ds", "y"]])  #'ds' (data) e 'y' (valor a ser previsto)

#datas futuras até 2034
future = model.make_future_dataframe(periods=11, freq="Y")
forecast = model.predict(future)  #previsões

#agrupa a previsão por ano
forecast["ano"] = forecast["ds"].dt.year
df_previsto = forecast.groupby("ano").agg({"yhat": "mean"}).reset_index()

#separa dados reais (até 203) previsões (a partir de 2024)
real = df_previsto[df_previsto["ano"] <= 2023]
previsao = df_previsto[df_previsto["ano"] > 2023]

fig = go.Figure()

#linha dos dados reais
fig.add_trace(go.Scatter(
    x=real["ano"], y=real["yhat"],
    mode="lines+markers",
    name="Dívida Real (2017–2023)",
    line=dict(color="green")
))

#linha das previsões futuras
fig.add_trace(go.Scatter(
    x=previsao["ano"], y=previsao["yhat"],
    mode="lines+markers",
    name="Previsão (2024–2034)",
    line=dict(color="blue", dash="dash")
))

fig.update_layout(
    title="📉 Previsão da Dívida Pública com Prophet (2017–2034)",
    xaxis_title="Ano",
    yaxis_title="Dívida Pública (Trilhões R$)",
    template="plotly_white",
    width=950,
    height=500
)

#mostrar todos os anos
fig.update_xaxes(
    tickmode="linear",
    tick0=2017,
    dtick=1
)

fig.show()


21:37:15 - cmdstanpy - INFO - Chain [1] start processing
21:37:20 - cmdstanpy - INFO - Chain [1] done processing


### **📊 Previsão da Dívida Pública (2017–2034)**
### 
**O gráfico apresenta a evolução da dívida pública brasileira entre 2017 e 2034.**

**Entre 2017 e 2023, observa-se uma tendência de crescimento contínuo, partindo de aproximadamente R$ 5,3 trilhões até cerca de R$ 8,9 trilhões.**

**A partir de 2024, as previsões indicam uma aceleração na dívida, ultrapassando R$ 14 trilhões em 2028 e podendo alcançar cerca de R$ 20 trilhões até 2034, conforme projeções do modelo Prophet.**

**Essa tendência reforça a necessidade de políticas fiscais sustentáveis para conter o avanço da dívida no longo prazo.**

**-------------------------------------------------------------------------------------------------------------------------------------------**

In [0]:
from prophet import Prophet
import pandas as pd
import plotly.graph_objects as go

df = df_baseagregada.toPandas().copy()

df["ano"] = df["ano"].astype(int)   #coluna "ano" inteiro
df["renuncia_total"] = df["renuncia_total"].astype(float)   #converte float

#filtro dados a partir de 2017
df = df[df["ano"] >= 2017].copy()

df["y"] = df["renuncia_total"] / 1e9 #converte para bihões
df["ds"] = pd.to_datetime(df["ano"], format="%Y")   #cria coluna de data exigida Prophet   

#cria e treina o modelo Prophet
model = Prophet()
model.fit(df[["ds", "y"]])   #'ds' (data) e 'y' (valor a ser previsto)

#datas futuras até 2034
future = model.make_future_dataframe(periods=11, freq="Y")
forecast = model.predict(future) #previsões

#agrupa a previsão por ano
forecast["ano"] = forecast["ds"].dt.year
df_previsto = forecast.groupby("ano").agg({"yhat": "mean"}).reset_index()

#separa dados reais (até 203) previsões (a partir de 2024)
real = df_previsto[df_previsto["ano"] <= 2023]
previsao = df_previsto[df_previsto["ano"] > 2023]

fig = go.Figure()

#linha dos dados reais
fig.add_trace(go.Scatter(
    x=real["ano"], y=real["yhat"],
    mode="lines+markers",
    name="Renúncia Real (2017–2023)",
    line=dict(color="green")
))

#linha das previsões futuras
fig.add_trace(go.Scatter(
    x=previsao["ano"], y=previsao["yhat"],
    mode="lines+markers",
    name="Previsão (2024–2034)",
    line=dict(color="blue", dash="dash")
))

fig.update_layout(
    title="📉 Previsão da Renúncia Fiscal com Prophet (2017–2034)",
    xaxis_title="Ano",
    yaxis_title="Renúncia Fiscal (Bilhões R$)",
    template="plotly_white",
    width=950,
    height=500
)

fig.update_xaxes(
    tickmode="linear",
    tick0=2017,
    dtick=1
)

fig.show()

21:41:20 - cmdstanpy - INFO - Chain [1] start processing
21:41:20 - cmdstanpy - INFO - Chain [1] done processing


### **📊 Previsão da Renúncia Fiscal (2017–2034)**

**O gráfico mostra a evolução da renúncia fiscal entre 2017 e 2034.**
**No período de 2017 a 2023, os valores oscilaram entre R$ 15 bilhões e R$ 40 bilhões, com aumentos mais expressivos após 2020.**

**As projeções para 2024 a 2034 indicam um crescimento gradual da renúncia, podendo atingir valores próximos a R$ 85 bilhões até 2034.**
**Esse aumento sugere atenção às políticas de incentivos fiscais e seus impactos na arrecadação pública ao longo da próxima década.**


**-------------------------------------------------------------------------------------------------------------------------------------------**

### Conclusão

**🔹 O projeto evidenciou o desafio de consolidar e interpretar dados públicos de forma clara e acessível. A integração dos dados de investimento, dívida e renúncia mostrou não só a complexidade da gestão financeira pública, mas também a carência de dados padronizados e completos. A previsão com Prophet, embora útil para indicar tendências, depende fortemente da qualidade dos dados históricos – um ponto crítico ao se trabalhar com fontes públicas.**

**🔹 As visualizações ajudaram a revelar desigualdades regionais nos investimentos e a evolução preocupante da dívida, que tende a crescer mais do que os aportes públicos. Essa disparidade levanta questões sobre a sustentabilidade fiscal e a eficácia da alocação de recursos no país.**

**🔹 Apesar das limitações técnicas e estruturais dos dados, o projeto reforçou a importância da transparência pública e mostrou como a ciência de dados pode ser uma ferramenta essencial para diagnósticos estratégicos, principalmente em contextos governamentais.**

**-------------------------------------------------------------------------------------------------------------------------------------------**